# Nuke button

In [4]:
# Delete all files and folders in the working directory. Use with caution!
import shutil
shutil.rmtree("/kaggle/working", ignore_errors=True)

# Code

## Git Clone

In [5]:
!git clone -q https://github.com/BinhPQ2/magi_functional.git

In [1]:
!pip install -qr /kaggle/working/magi_functional/requirements_kaggle.txt

## Load MAGI model

In [2]:
from PIL import Image
import numpy as np
from transformers import AutoModel
import torch

model = AutoModel.from_pretrained("ragavsachdeva/magiv2", trust_remote_code=True).cuda().eval()

config.json:   0%|          | 0.00/13.4k [00:00<?, ?B/s]

configuration_magiv2.py:   0%|          | 0.00/1.70k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ragavsachdeva/magiv2:
- configuration_magiv2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modelling_magiv2.py:   0%|          | 0.00/34.0k [00:00<?, ?B/s]

utils.py:   0%|          | 0.00/16.4k [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
VisionEncoderDecoderModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

## Save model to local

In [ ]:
# Save model to directory:
model.save_pretrained("./magi_saved_model_directory/")

In [ ]:
# Load model from directory:
model = AutoModel.from_pretrained("./magi_saved_model_directory/", trust_remote_code=True).cuda().eval()

In [ ]:
####################### OR ###########################
# Cache model to directory:
# model_1 = AutoModel.from_pretrained("ragavsachdeva/magiv2", cache_dir="./magi_saved_model_directory/", trust_remote_code=True).cuda().eval()

# Code

## Create raw and character/names list

In [9]:
import os
import re
from pathlib import Path

def create_chapter_pages_and_character_bank(manga_folder, character_folder):
    # Create lists for chapter pages and character bank
    chapter_pages = []
    character_bank = {
        "images": [],
        "names": []
    }

#     Iterate through manga images to create chapter_pages
    for image_file in os.listdir(manga_folder):
        if image_file.endswith(('.png', '.jpg', '.jpeg')):  # Check for image file extensions
            # Extract the page number using regex
            match = re.search(r'p(\d+)', image_file)
            if match:
                page_number = int(match.group(1))  # Convert to integer for sorting
                chapter_pages.append((page_number, image_file))  # Store as tuple (page_number, image_file)
            else:
                page_number = image_file.rsplit(".", 1)[0]
                chapter_pages.append((page_number, image_file))  # Store as tuple (page_number, image_file)

    # Sort chapter pages by page number
    chapter_pages.sort(key=lambda x: x[0])
    chapter_pages = [os.path.join(manga_folder, img[1]) for img in chapter_pages]  # Extract just the filenames after sorting

    # Iterate through character images to create character bank
    for char_image_file in os.listdir(character_folder):
        if char_image_file.endswith(('.png', '.jpg', '.jpeg')):  # Check for image file extensions
            # Split the filename to extract character name
            char_name = char_image_file.split('_')[0]  # Get the part before the underscore
            character_bank["images"].append(os.path.join(character_folder, char_image_file))
            character_bank["names"].append(char_name)
    return chapter_pages, character_bank

# Define your folders
# One Piece
# manga_folder = Path("/kaggle/working/magi_functional/data_test/personal_data/One_Piece/raw_manga")
# character_folder = Path("/kaggle/working/magi_functional/data_test/personal_data/One_Piece/character")

# Ruri Dragon
manga_folder = Path("/kaggle/working/magi_functional/data_test/personal_data/Magi/colorization")
character_folder = Path("/kaggle/working/magi_functional/data_test/personal_data/Magi/character")

# Get chapter pages and character bank
chapter_pages_original, character_bank_original = create_chapter_pages_and_character_bank(manga_folder, character_folder)

chapter_pages_test = chapter_pages_original[:]
character_bank_test = character_bank_original

# Print the results (for debugging)
print("Chapter Pages:")
print(chapter_pages_test)

print("\nCharacter Bank:")
print(character_bank_test)

Chapter Pages:
['/kaggle/working/magi_functional/data_test/personal_data/Magi/colorization/001.png', '/kaggle/working/magi_functional/data_test/personal_data/Magi/colorization/002.png', '/kaggle/working/magi_functional/data_test/personal_data/Magi/colorization/003.png', '/kaggle/working/magi_functional/data_test/personal_data/Magi/colorization/004.png', '/kaggle/working/magi_functional/data_test/personal_data/Magi/colorization/005.png', '/kaggle/working/magi_functional/data_test/personal_data/Magi/colorization/006.png', '/kaggle/working/magi_functional/data_test/personal_data/Magi/colorization/007.png', '/kaggle/working/magi_functional/data_test/personal_data/Magi/colorization/008.png', '/kaggle/working/magi_functional/data_test/personal_data/Magi/colorization/009.png', '/kaggle/working/magi_functional/data_test/personal_data/Magi/colorization/010.png', '/kaggle/working/magi_functional/data_test/personal_data/Magi/colorization/011.png', '/kaggle/working/magi_functional/data_test/person

## Process (OCR → Transcript)

In [10]:
import os
import json

def read_image(path_to_image):
    with open(path_to_image, "rb") as file:
        image = Image.open(file).convert("L").convert("RGB")
        image = np.array(image)
    return image

# chapter_pages = ["page1.png", "page2.png", "page3.png" ...]
# character_bank = {
#     "images": ["char1.png", "char2.png", "char3.png", "char4.png" ...],
#     "names": ["Luffy", "Sanji", "Zoro", "Ussop" ...]
# }


chapter_pages = [read_image(x) for x in chapter_pages_test]
character_bank = character_bank_test.copy()
character_bank["images"] = [read_image(x) for x in character_bank_test["images"]]

with torch.no_grad():
    per_page_results = model.do_chapter_wide_prediction(chapter_pages, character_bank, use_tqdm=True, do_ocr=True)

result_dir = "/kaggle/working/result"
json_output_dir = f"{result_dir}/json_results"
result_image_output_dir = f"{result_dir}/image_results"
transcript_output_dir = f"{result_dir}/transcript.txt"
os.makedirs(json_output_dir, exist_ok=True)  # Create the directory if it doesn't exist  
os.makedirs(result_image_output_dir, exist_ok=True)  # Create the directory if it doesn't exist  

transcript = []
for i, (image, page_result) in enumerate(zip(chapter_pages, per_page_results)):
    model.visualise_single_image_prediction(image, page_result, os.path.join(result_image_output_dir, f"page_{i}.png"))
    # Save page_result to JSON
    json_file_path = os.path.join(json_output_dir, f"page_{i}_result.json")  # Create full file path
    with open(json_file_path, 'w') as json_file:
        json.dump(page_result, json_file, indent=4)  # Save with pretty printing

    speaker_name = {
        text_idx: page_result["character_names"][char_idx] for text_idx, char_idx in page_result["text_character_associations"]
    }
    for j in range(len(page_result["ocr"])):
        if not page_result["is_essential_text"][j]:
            continue
        name = speaker_name.get(j, "unsure") 
        transcript.append(f"<{name}>: {page_result['ocr'][j]}")
    transcript.append("---------------------------------------------")
with open(transcript_output_dir, "w") as fh:
    for line in transcript:
        fh.write(line + "\n")

print("\n\nDone you WEEEEB!")

100%|██████████| 3/3 [00:04<00:00,  1.47s/it]


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/660a5052b5ab425faad6374739829ee9-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/660a5052b5ab425faad6374739829ee9-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12986 COLUMNS
At line 45059 RHS
At line 58041 BOUNDS
At line 59687 ENDATA
Problem MODEL has 12981 rows, 1645 columns and 27137 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 168.945 - 0.02 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 9527 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 8918 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 8703 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 8622 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened

100%|██████████| 3/3 [00:19<00:00,  6.46s/it]




Done you WEEEEB!


# Zip file

## Legacy

In [ ]:
# json_output_dir = "/kaggle/working/json_results"
# result_image_output_dir = "/kaggle/working/image_results"

# # zip
# json_output_dir = "/kaggle/working/json_results"
# !zip -rj /kaggle/working/json.zip {json_output_dir}/*.json
# # download
# !cd /kaggle/working/
# display(FileLink('json.zip'))

## Download separate files

### Download json

In [5]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(download_file_name, source_path):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip -rj {zip_name} {source_path}"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))

download_file("json_results", f"{json_output_dir}/*.json")

/kaggle/working/json_results.zip

### Download image results

In [6]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(download_file_name, source_path):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip -rj {zip_name} {source_path}"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))

download_file("image_results", result_image_output_dir)

/kaggle/working/image_results.zip

### Download transcript

In [7]:
import os
os.chdir('/kaggle/working/')
display(FileLink('transcript.txt'))

/kaggle/working/transcript.txt

## Download all

In [16]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(download_file_name, source_path):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip -r {zip_name} {source_path}"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))
    

!cp -r result_dir /
download_file("result", result_dir)

/kaggle/working/result.zip